In [17]:
import sys
sys.path.append('..')
import constants.constants as const
import constants.file_handler_constants as fh
from constants.attraction_constants import *

from packages.attraction.Attraction import *
from packages.file_handler_package.file_handler import *

import os
import glob
import time
import pandas as pd
import ast
import json
import requests
import ast

In [21]:
path_to_res_merge_attraction_phuket = os.path.join(fh.STORE_FILL_ATTRACTION_TAG_SCORE, 'res_merge_attraction', 'res_merge_attraction_Phuket.csv')
merge_attraction_phuket_df = pd.read_csv(path_to_res_merge_attraction_phuket)

merge_attraction_phuket_df = merge_attraction_phuket_df.fillna('')
merge_attraction_phuket_df

,name,type,description,latitude,longitude,imgPath,phone,website,openingHour,address,...,Zoo,Waterpark,Hiking,Museum,Riverside,NightLife,Family,Kid,Landmark,Forest
0,เลิฟ ไดวิ่ง ภูเก็ต,['การดำน้ำลึกและการดำน้ำตื้น'],,7.893529,98.297264,['https://dynamic-media-cdn.tripadvisor.com/me...,,https://th.tripadvisor.com/Attraction_Review-g...,"{'จันทร์': '12:00-21:00', 'อังคาร': '12:00-21:...",V7VW+CW6 ตำบลป่าตอง อำเภอกะทู้ ภูเก็ต,...,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.0
1,Andaman Sea Kayaks,['การพายเรือคายัคและการพายเรือแคนู'],,7.888660,98.380020,['https://dynamic-media-cdn.tripadvisor.com/me...,,https://th.tripadvisor.com/Attraction_Review-g...,"{'จันทร์': '08:00-20:00', 'อังคาร': '08:00-20:...",V9QJ+F26 ตำบล วิชิต อำเภอเมืองภูเก็ต ภูเก็ต,...,0.0,0.0,0.0,0.0,0.2,0.0,0.7,0.5,0.2,0.1
2,The Junk Liveaboards,['การดำน้ำลึกและการดำน้ำตื้น'],,7.859957,98.352960,['https://dynamic-media-cdn.tripadvisor.com/me...,,https://th.tripadvisor.com/Attraction_Review-g...,"{'จันทร์': '09:00-17:00', 'อังคาร': '09:00-17:...",V953+X5P ตำบล ฉลอง อำเภอเมืองภูเก็ต ภูเก็ต,...,0.0,0.0,0.2,0.1,0.2,0.2,0.5,0.4,0.2,0.2
3,ACDC Diving,['การดำน้ำลึกและการดำน้ำตื้น'],,7.840755,98.345090,['https://dynamic-media-cdn.tripadvisor.com/me...,,https://th.tripadvisor.com/Attraction_Review-g...,"{'จันทร์': '08:00-21:00', 'อังคาร': '08:00-21:...",R8RW+824 ตำบล ฉลอง อำเภอเมืองภูเก็ต ภูเก็ต,...,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.1,0.0,0.0
4,พาราไดส์ ไดวิ่ง เอเซีย,['การดำน้ำลึกและการดำน้ำตื้น'],,8.087546,98.297670,['https://dynamic-media-cdn.tripadvisor.com/me...,,https://th.tripadvisor.com/Attraction_Review-g...,"{'จันทร์': '10:00-16:00', 'อังคาร': '10:00-16:...",37QX+237 ตำบลสาคู อำเภอถลาง ภูเก็ต,...,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.5,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,คันทรีบาร์ บางลา,['บาร์/คลับ'],,7.894073,98.297030,['https://dynamic-media-cdn.tripadvisor.com/me...,,https://th.tripadvisor.com/Attraction_Review-g...,"{'จันทร์': '19:00-02:00', 'อังคาร': '19:00-02:...",V7VW+JRC ตำบลป่าตอง อำเภอกะทู้ ภูเก็ต,...,0.0,0.0,0.0,0.0,0.0,0.7,0.0,0.0,0.0,0.0
373,Kid's Club,['เกมและแหล่งรวมความบันเทิง'],ที่ฮอลิเดย์อินน์รีสอร์ทภูเก็ตหาดไม้ขาวมี Kid's...,7.996620,98.349390,['https://dynamic-media-cdn.tripadvisor.com/me...,,https://th.tripadvisor.com/Attraction_Review-g...,{},"81 หมู่ 3 ฮอลิเดย์ อินน์ รีสอร์ต ภูเก็ต, ไม้ขา...",...,0.0,0.0,0.0,0.0,0.0,0.8,0.7,0.9,0.0,0.0
374,Yellow Clock Tower,['สถานที่สำคัญ/จุดที่น่าสนใจ'],,7.883170,98.395370,['https://dynamic-media-cdn.tripadvisor.com/me...,,https://th.tripadvisor.com/Attraction_Review-g...,{},"Intersection of Phuket Rd and, Phangnga Rd, Mu...",...,0.0,0.0,0.1,0.0,0.1,0.4,0.1,0.1,0.4,0.0
375,The Garden Phuket Luxury Mall,['ศูนย์การค้า'],เดอะ การ์เด้น : ภูเก็ต ลักชัวรี่ มอลล์ ศูนย์กา...,8.015935,98.337840,['https://dynamic-media-cdn.tripadvisor.com/me...,,https://th.tripadvisor.com/Attraction_Review-g...,"{'จันทร์': '09:00-18:00', 'อังคาร': '09:00-18:...","124/1 หมู่ 1 ถ.เทพกษัตรี, เมืองภูเก็ต, จังหวัด...",...,0.0,0.0,0.0,0.2,0.0,0.0,0.3,0.3,0.2,0.0


In [19]:
def create_payload(row: pd.Series) -> dict:

    openingHour_list_payload = [{ "day": key, "openingHour": val } for key, val in ast.literal_eval(row['openingHour']).items()]

    attractionTagFields_dict_payload = {cur_tag:row[cur_tag] for cur_tag in ATTRACTION_TAG_SCORE}

    payload = {
        'name': row['name'],
        "type": ast.literal_eval(row['type']),
        "description": row['description'],
        "latitude": row['latitude'],
        "longitude": row['longitude'],
        "imgPath": ast.literal_eval(row['imgPath']),
        "phone": row['phone'],
        "website": row['website'],
        "openingHour": openingHour_list_payload.copy(),
        "attractionTag": {
            "attractionTagFields": attractionTagFields_dict_payload.copy()
        },
        "location": {
            "address": row['address'],
            "province": row['province'],
            "province_code": row['province_code'],
            "district": row['district'],
            "district_code": row['district_code'],
            "subDistrict": row['subDistrict'],
            "sub_district_code": row['sub_district_code']
        },
        "rating": {
            "score": row['score'],
            "ratingCount": row['ratingCount']
        }
    }

    return payload.copy()

In [20]:
for Idx, cur_row in merge_attraction_phuket_df.iterrows():

    cur_payload = create_payload(row=cur_row)

    try:
        API_ENDPOINT = f"{os.environ['TRIPWEAVER_API']}/api/attraction/create"
        post_request = requests.post(url=API_ENDPOINT, json=cur_payload)
        print(post_request.text)

    except Exception as e:
        print("POST api failed at --> ", cur_payload['name'])
        print(e)

{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{"message":"Create Data attraction"}
{